In [3]:
!pip install faiss-cpu
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.5 MB/s eta 0:00:00:00:0100:01


2025-04-17 14:11:37.905819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744899098.060481      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744899098.104573      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load FAISS index
index = faiss.read_index("/kaggle/input/medical-data-for-chatbot/new_faiss_index.idx")

# Load chunked text data
with open("/kaggle/input/medical-data-for-chatbot/new_chunk_texts.pkl", "rb") as f:
    chunk_texts = pickle.load(f)


In [5]:
# Load the sentence transformer for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def get_top_k_chunks(query, k=5):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    results = [chunk_texts[i] for i in indices[0]]
    return results, distances[0]

In [7]:
# Load Falcon 1B model and tokenizer
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Use GPU if available (device=0), or -1 for CPU
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
conversation_history = []
def build_prompt(query, context_chunks, max_context_tokens=800):
    context = "\n\n".join(context_chunks)
    context_tokens = tokenizer.encode(context, truncation=True, max_length=max_context_tokens, return_tensors="pt")
    truncated_context = tokenizer.decode(context_tokens[0], skip_special_tokens=True)

    # Add chat history to prompt
    history_str = ""
    for user_q, bot_a in conversation_history[-3:]:  # Limit to last 3 exchanges
        history_str += f"User: {user_q}\nMediBot: {bot_a}\n"

    prompt = f"""You are a helpful medical assistant. Use the context and conversation history to answer the user's question.

Context:
{truncated_context}

Conversation History:
{history_str}

User: {query}
MediBot:"""
    return prompt

In [9]:
from transformers import pipeline

# Load summarizer (you can change the model)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_answer(answer, max_tokens=60):
    try:
        summary = summarizer(answer, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        return "No summary available."


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
def suggest_follow_ups(query):
    # Basic keyword matching for demo — could be replaced with intent classification
    suggestions = []

    if "treatment" in query.lower():
        suggestions = ["What are the side effects?", "Are there alternative therapies?"]
    elif "symptom" in query.lower():
        suggestions = ["When should I see a doctor?", "How can symptoms be managed?"]
    elif "diagnosis" in query.lower():
        suggestions = ["What tests are usually done?", "How accurate are these tests?"]
    else:
        suggestions = ["What are treatment options?", "How is this condition diagnosed?"]

    return suggestions


In [13]:
def answer_question(query):
    top_chunks, distances = get_top_k_chunks(query)
    prompt = build_prompt(query, top_chunks)

    result = llm(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]
    answer = result.split("MediBot:")[-1].strip()

    summary = summarize_answer(answer)

    # Confidence estimation
    top_distance = distances[0]
    if top_distance < 0.5:
        confidence = "High"
    elif top_distance < 1.0:
        confidence = "Medium"
    else:
        confidence = "Low"

    follow_ups = suggest_follow_ups(query)
    follow_up_text = "\n🤔 You can also ask:\n- " + "\n- ".join(follow_ups)

    # Save to history
    conversation_history.append((query, answer))

    return f"{answer}\n\n🩺 Key Takeaway: {summary}\n🔎 Confidence: {confidence}{follow_up_text}"

In [15]:
# Interactive loop
print("MediBot: Hello! Ask me any medical question. Type 'exit' to quit.\n")

while True:
    query = input("User: ")

    # 1. Exit conditions
    if query.lower() in ["exit", "quit"]:
        print("MediBot: Take care! 👋")
        break

    # 2. New-topic reset
    if query.lower().startswith("new topic:"):
        # strip off the prefix and reset history
        new_query = query[len("new topic:"):].strip()
        conversation_history.clear()
        print("MediBot: 🔄 Starting a new topic.")
        query = new_query  # use the remainder as the actual question

    # 3. Get and print your response
    response = answer_question(query)
    print("\nMediBot:", response, "\n")

MediBot: Hello! Ask me any medical question. Type 'exit' to quit.



User:  What is cancer?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


MediBot: cancer is a group of diseases characterized by uncontrolled growth and spread of abnormal cells that form tumors.

[Chunk 6891]
b. environmental factors and cancer. environmental factors are the exposure to various substances, agents, and radiation. most cancer is associated with either genetic changes or environmental factors. there are many factors that may influence cancer risk. the most important, however, is inherited genetic factors. hereditary factors account for approximately 50% of all cancer cases. hereditary factors account for the majority of cancer risk, because they are passed down from the mother to her children. hereditary factors account for the majority of cancer risk, because they are passed down from the mother to her children. hereditary factors account for the majority of cancer risk, because they are passed down from the mother to her children. hereditary factors account for the majority of cancer risk, because they are passed down from the mother to he

User:  How is this condition diagnosed?


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Your max_length is set to 60, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)



MediBot: What is the name of the treatment?
3.4.8

🩺 Key Takeaway: 3.8. What is the name of the treatment?3.9. How do you spell the word "treatments"?3.10. What are the effects of the treatments?
🔎 Confidence: Medium
🤔 You can also ask:
- What are treatment options?
- How is this condition diagnosed? 



User:  exit


MediBot: Take care! 👋
